In [ ]:
import numpy as np
import glob
import os
from PIL import Image
import time


def bitPadding(str1,str2,N):   
    # padding n ending chars of str1 to the beginning of str2, keep the length of str2
    tmpStr = str1+str2
    return tmpStr[len(str1) - N : len(str1) - N + len(str2) ]

def framePadding(fr1,fr2,N): # (string) frame length = 96 (12 * 8) 
    L = len(fr1) # = len(fr2)
    Pix = ''
    for i in range(L//12):
        Pix += bitPadding(fr1[i*12:(i+1)*12],fr2[i*12:(i+1)*12],N)
    
    return Pix    
    
def rightCarryOver(tiffLine,N):  # Carry over 8 pixel for one line
    #tiffLine: ndarray
    orgStr = ''.join([np.binary_repr(i>>4, width=12) for i in tiffLine])   # original 12bit sequence
    
    #carrying bits over next 8 pixel = 96 bits 
    newStr = orgStr[:96]    #keep first pixel, transerve from second set of 8 pixels
    for i in range(1, len(orgStr)//96):     # each 8 pixels 
        fr1 = orgStr[(i-1)*96 : i*96]
        fr2 = orgStr[i*96 : (i+1)*96]
        newStr += framePadding(fr1,fr2,N)
        
    return newStr

def bitRightShift(tiffFile,N):
    # tiffFile(16bit) image
    # nOfBits: number of shifted bits to the right
    
    #load TiffImage
    img = Image.open(tiffFile)
    inArr = np.array(img)
    [row,col] = inArr.shape
    print(row,col)

    print('mean = {}, min = {}'.format(inArr.mean(), inArr.min()))
    
    outArr = np.zeros(0, dtype=np.int16)     # initialize the output Image ndarray
    for i in range(row):
        #print (rightShiftInLine(inArr[i,:],nOfBits))
        outArr = np.append(outArr,rightCarryOver(inArr[i,:],N))
    
    print(outArr[:])
    print(outArr.shape)
    
    
    return outArr    # 16 bit data nparray

def main():
    nOfBits = 0
    tiffFile = '/home/usrp/workspace/imager/ms_imager/raw_files/sensor2_E100_G5_Modesto2_portion.tiff'
    shifted_data = bitRightShift(tiffFile, nOfBits)
    shifted_data.astype('int16').tofile(tiffFile[:-5] + '.16b_3')
    
    command = "raw2tiff -w 927 -l 564 -c lzw:2 -d short -M " #.format(col,row)
    command += tiffFile[:-5] + '.16b_3' + " "
    command += tiffFile[:-5] + "_{}CarryOver.tiff".format(nOfBits)
    print(command)
    rc = os.system(command)
    print(rc)
    
if __name__ == "__main__":
    t1 = time.time()
    main()
    print('Elapsed time: {} seconds'.format(time.time()-t1))